# HW1 - Brian Flores

## Q2

Return orders placed on the day before the last day of the month

**Tables involved:** Sales.Orders table, using TSQLV4. Sales.Order table, using <span style="color:rgb(33, 33, 33);font-family:Menlo, Monaco, &quot;Courier New&quot;, monospace;font-size:12px;white-space:pre;">Northwinds2022TSQLV7.</span>

**Similar goal output:**

```
orderid     orderdate  custid      empid
----------- ---------- ----------- -----------
10268       2020-07-30 33          8
10294       2020-08-30 65          4
10342       2020-10-30 25          4
10368       2020-11-29 20          2
10398       2020-12-30 71          2
10430       2021-01-30 20          4
10431       2021-01-30 10          4
10459       2021-02-27 84          4
10520       2021-04-29 70          7
10521       2021-04-29 12          8
...

```

_(27 row(s) affected)_

* * *

## **<span style="font-family:georgia, palatino, serif;margin:0px;padding:0px;border:0px;outline:0px;font-size:14pt;text-shadow:none;">Proposition, Table, Columns and the Predicate.</span>**

The question is based on the what predicate to use in order to check if a order is placed before the last day of the month. This **Problem** can be extended to any table that involves a date. The **columns** to be affected By this are orderdate and any other columns that which to be included as a result of the query. There are two ways to get this comparison to work, this is the **predicate** of the solution that will enable us to get the right result.

#### Using the End of month function

We can invoke the `EOMONTH(date)` function to return what the end of the month would be. Then subtract by one day back to get the second to last day of any month using <span style="color:rgb(163, 21, 21);font-family:Menlo, Monaco, Consolas, &quot;Droid Sans Mono&quot;, &quot;Courier New&quot;, monospace, &quot;Droid Sans Fallback&quot;;font-size:12px;">DATEADD(day, -1, date)</span>.

#### Using the Date arithmetic

Peform a complex manipulation with `DATEDIFF(type, firstDate, secondDate)` and `DATEADD(type, units, date)`. To do the latter operation we would have to first subtract how many units from a base date whos date is the last day of the month. Then you use that value to add it to the base date to get the date to compare your current date with. Finally subtract by one day to get the second to last day.

### **<u>Prefered method</u>**

The former solution is both simple and readable. This solution also does not require the developer to know thier date trivia, therefore making it an accesible solution.

**Note**

The 3rd edition changes the question to strictly the last dat of the month. The question above uses the 4th edition so you will find different number of records if running on different databases.

In [ ]:
-- Original Query for TSQLV4

USE TSQLV4;

SELECT orderid, orderdate, custid, empid
FROM Sales.Orders
WHERE orderdate = DATEADD(day, -1, EOMONTH(orderdate));


-- New Query for Northwinds

USE Northwinds2022TSQLV7;

SELECT OrderId, OrderDate, CustomerId, EmployeeId
FROM Sales.[Order]
WHERE OrderDate = DATEADD(day, -1, EOMONTH(OrderDate));


## Q10

Return for each customer the customer ID and region, sort the rows in the output by region, having NULLs sort last (after non-NULL values). Note that the default in T-SQL is that NULLs sort first.

**Tables involved:** Sales.Customers table using TSQLV4. Sales.Customer table, using      <span style="color:rgb(33, 33, 33);font-family:Menlo, Monaco, &quot;Courier New&quot;, monospace;font-size:12px;white-space:pre;">Northwinds2022TSQLV7.</span>

**Desired output:**

```
custid      region
----------- ---------------
55          AK
10          BC
42          BC
45          CA
37          Co. Cork
33          DF
71          ID
38          Isle of Wight
46          Lara
78          MT
...
1           NULL
2           NULL
3           NULL
4           NULL
5           NULL
6           NULL
7           NULL
8           NULL
9           NULL
11          NULL
...

```

_(91 row(s) affected)_

* * *

## **<span style="font-family:georgia, palatino, serif;margin:0px;padding:0px;border:0px;outline:0px;font-size:14pt;text-shadow:none;">Proposition, Table, Columns and the Predicate.</span>**

The **proposition** of this question is that we are trying to change the presentation of a sorting operation. Our instinctive approach is to simply use the orderby clause to sort based on region. However, T-SQL defaults to null having a higher precedence as such will bubble to the top. Instead what if we were to detect when a null apperead and gave it less precedence. As a comparison, in java programs we may want to create a our custom comparison to sort objects in a collection. We can achieve a similar result by using Case expressions. The **table** being affected is the customers table, however learning this technique can help us adapt any sorting technique to a different table. The **columns** being affected are the custid and orderdate. As hinted before the **predicate** for this solution involves some function that gives a result in this case a case expression or a function that choses.

#### Case Expressions

Are like mini procedures that evaulate on branch logic. We can assign nulls to a higher value meaning it will choose it last and give non null items higher precedence by using a lower number.

### <span style="font-family:-apple-system, BlinkMacSystemFont, sans-serif;font-size:16.38px;"><b><u>Analogy</u></b></span>

### <span style="font-size:14px;font-family:-apple-system, BlinkMacSystemFont, sans-serif;">To make this understandable lets use an analogy on strings if we compare strings character by character we will be able to know whether a string deserves higher precdence in the same way by using nullability it is replaced with a higher value like z.&nbsp;</span> 

<span style="font-size:14px;font-family:-apple-system, BlinkMacSystemFont, sans-serif;"><i><br></i></span>

<span style="font-size:14px;font-family:-apple-system, BlinkMacSystemFont, sans-serif;"><b><u><i>Alternative solution</i></u></b></span>

Upon asking gpt "How can I make this better", it gave a brilliant suggestion. Instead of manually choosing what we should sort by we can instead default to a value whenever Null is achieved, we can do this with the function `ISNULL(arg1, arg2)` where arg2 is what should be returned if arg1 happens to be null. Instead we could `ORDER BY ISNULL(region, 'ZZZZZZZZZZZZZZZZ')`. This function along with `COALESCE` is also explained in the authors book (Ben-Gan, 55). One may ask when to chose one over the other, `COALESCE` is standard while `ISNULL` is not. One is more readable than the other which is why others may remember to choose it more.

In [ ]:
-- Original Query for TSQLV4

USE TSQLV4;

SELECT custid, region
FROM Sales.Customers
ORDER BY CASE WHEN region IS NULL THEN 1 ELSE 0 END, region;


-- New Query for Northwinds

USE Northwinds2022TSQLV7;

SELECT customerId, CustomerRegion
FROM Sales.Customer
ORDER BY COALESCE(CustomerRegion, 'ZZZZZZZZZZ');


# Propositions
Translating queries to problems.

* * *

### **13\. LIKE Predicate**

## 13.1

Using the HumanResources.Employee table search for employees whose last name start with the letter d. Display the result with thier respective id and thier last name. Where the search is case insensitive.

_**Tables involved:**_ HumanResources.Employees table

In [ ]:
-- 13.1

USE TSQLV4;

SELECT empid, lastname
FROM HR.Employees
WHERE lastname LIKE N'D%';

USE Northwinds2022TSQLV7;

SELECT EmployeeId, EmployeeLastName
FROM HumanResources.Employee
WHERE EmployeeLastName LIKE N'D%';

## 13.2

Using the HumanResources.Employee table search for employees whose last name has e as thier second character in thier name. Display the result with thier respective id and thier last name. Where the search is case insensitive.

**Tables involved:** HumanResources.Employee table

In [ ]:
-- 13.2

USE TSQLV4;

SELECT empid, lastname
FROM HR.Employees
WHERE lastname LIKE N'_e%';


USE Northwinds2022TSQLV7;

SELECT EmployeeId, EmployeeLastName
FROM HumanResources.Employee 
WHERE EmployeeLastName LIKE N'_e%';

## 13.3

Using the HumanResources.Employee table search for employees whose last name starts with the letter a or b or c. Display the result with thier respective id and thier last name. Where the search is case insensitive.

**Tables involved:** HumanResources.Employee table

In [ ]:
-- 13.3
USE TSQLV4;

SELECT empid, lastname
FROM HR.Employees
WHERE lastname LIKE N'[ABC]%';

USE Northwinds2022TSQLV7

SELECT EmployeeId, EmployeeLastName
FROM HumanResources.Employee
WHERE EmployeeLastName LIKE N'[ABC]%';

## 13.4

Using the HumanResources.Employee table search for employees whose last name starts with the letter a or b or c or d or e. Display the result with thier respective id and thier last name. Where the search is case insensitive.

Tables involved: HumanResources.Employee table



In [ ]:
-- 13.4
USE TSQLV4;

SELECT empid, lastname
FROM HR.Employees
WHERE lastname LIKE N'[A-E]%';


USE Northwinds2022TSQLV7;

SELECT EmployeeId, EmployeeLastName
FROM HumanResources.Employee
WHERE EmployeeLastName LIKE N'[A-E]%';

## 13.5

Using the HumanResources.Employee table search for employees whose last name does not start with the letter a or b or c or d or e. Display the result with thier respective id and thier last name. Where the search is case insensitive.

**Tables involved:** HumanResources.Employee table

In [ ]:
-- 13.5

USE TSQLV4;

SELECT empid, lastname
FROM HR.Employees
WHERE lastname LIKE N'[^A-E]%';

USE Northwinds2022TSQLV7;

SELECT EmployeeId, EmployeeLastName
FROM HumanResources.Employee
WHERE EmployeeLastName LIKE N'[^A-E]%';


* * *

## 15 Querying Metadata

### 15.1

### Catalog Views

Within the NorthWinds database display all the tables and their respective schemas. Display the schema table column header in a readable format.


In [ ]:
-- 15.1

USE TSQLV4;

SELECT SCHEMA_NAME(schema_id) AS table_schema_name, name AS table_name
FROM sys.tables;

USE Northwinds2022TSQLV7;

SELECT SCHEMA_NAME(schema_id) AS table_schema_name, name AS table_name
FROM sys.tables;

### 15.2 

Within the TSQLV4 database display, display metadata (with respect to its columns) about the table `Sales.Order`. The metadata needed are the columns name. Thier repspective data type, thier max-length of bytes of memory that the attribute may take up. For columns that involve characters print out what specific collation it is using and leaving those who dont use one as null. Finally, append a column that tells you whether a column can accept null values by a matter of boolean. Use `sys.columns` that tells you aboult columns in the database, within the table you have to invoke the `OBJECT_ID(N'Schema.Table')` function to know how to pick the right table.


In [ ]:
-- 15.2
USE TSQLV4;

SELECT 
  name AS column_name,
  TYPE_NAME(system_type_id) AS column_type,
  max_length,
  collation_name,
  is_nullable
FROM sys.columns
WHERE object_id = OBJECT_ID(N'Sales.Orders');

USE Northwinds2022TSQLV7;

SELECT 
  name AS column_name,
  TYPE_NAME(system_type_id) AS column_type,
  max_length,
  collation_name,
  is_nullable
FROM sys.columns
WHERE object_id = OBJECT_ID(N'Sales.Order');

## 15.3  

Query the schema `INFORMATION_SCHEMA` which holds information (objects) about your database (tables, procedures, constraints, etc...). Within this schema find all the tables that was created by a user. You will need to use the table `TABLES`. Output the table and thier respective schemas.


In [ ]:
-- 15.3

USE TSQLV4;

SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = N'BASE TABLE';

USE Northwinds2022TSQLV7;
SELECT TABLE_SCHEMA, TABLE_NAME
FROM INFORMATION_SCHEMA.TABLES
WHERE TABLE_TYPE = N'BASE TABLE';

## 15.4  

Query the schema `INFORMATION_SCHEMA` . Within this schema Find the table `Sales.Order` and find out metadata about its columns. You will need to use the table `COLUMNS`. Output the Columns name, its data type, its character limit (if column is a character related type), and whether it can be null.


In [ ]:
-- 15.4

USE TSQLV4;

SELECT 
  COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, 
  COLLATION_NAME, IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = N'Sales'
  AND TABLE_NAME = N'Orders';

-- 15.4
USE Northwinds2022TSQLV7;

SELECT 
  COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH, 
  COLLATION_NAME, IS_NULLABLE
FROM INFORMATION_SCHEMA.COLUMNS
WHERE TABLE_SCHEMA = N'Sales'
  AND TABLE_NAME = N'Order';

###  15.5

### System Stored Procedures and Functions

Invoke a system procedure that will display all the tables and views in any database.


In [ ]:
-- 15.5

USE TSQLV4;
EXEC sys.sp_tables;

USE Northwinds2022TSQLV7;
EXEC sys.sp_tables;

### 15.6
Invoke a system procedure to obtain all relevant metadata regarding the table `Sales.Order`.


In [ ]:
-- 15.6

USE TSQLV4;

EXEC sys.sp_help
  @objname = N'Sales.Orders';

USE Northwinds2022TSQLV7;
EXEC sys.sp_help
  @objname = N'Sales.Order';

## 15.7
Invoke a system procedure to obtain all information about the columns of the schema `Sales.Order.`

In [ ]:
-- 15.7

USE TSQLV4;

EXEC sys.sp_columns
  @table_name = N'Orders',
  @table_owner = N'Sales';

USE Northwinds2022TSQLV7;

EXEC sys.sp_columns
  @table_name = N'Order',
  @table_owner = N'Sales';

## 15.8
Invoke a system procedure to obrain information on constraints with respect to the schema `Sales.Order`.



In [ ]:
-- 15.8

USE TSQLV4;
EXEC sys.sp_helpconstraint
  @objname = N'Sales.Orders';

USE Northwinds2022TSQLV7;
EXEC sys.sp_helpconstraint
  @objname = N'Sales.Order';

## 15.8
Return what product or level of SQL server you are using.



In [ ]:
-- 15.8
SELECT 
  SERVERPROPERTY('ProductLevel');


## 15.9
Invoke the `DATABASEPROPERTYEX` function to return collation information about the database `Northwinds2022TSQLV7`.

In [ ]:
-- 15.9

USE TSQLV4;
SELECT
  DATABASEPROPERTYEX(N'TSQLV4', 'Collation');

USE Northwinds2022TSQLV7;
SELECT
  DATABASEPROPERTYEX(N'Northwinds2022TSQLV7', 'collation');

## 15.9
Check whether or not the table `Sales.Order` has a primary key.

In [ ]:
-- 15.9

USE TSQLV4;

SELECT 
  OBJECTPROPERTY(OBJECT_ID(N'Sales.Orders'), 'TableHasPrimaryKey');

USE Northwinds2022TSQLV7;

SELECT 
  OBJECTPROPERTY(OBJECT_ID(N'Sales.Order'), 'TableHasPrimaryKey');


## 15.10
Return whether the column shipCountry in the table `Sales.Order` is allowed to insert nulls.


In [ ]:
-- 15.10

USE TSQLV4;

SELECT
  COLUMNPROPERTY(OBJECT_ID(N'Sales.Orders'), N'shipcountry', 'AllowsNull');

USE Northwinds2022TSQLV7;

SELECT
  COLUMNPROPERTY(OBJECT_ID(N'Sales.Order'), N'ShipToCountry', 'AllowsNull');